<a href="https://colab.research.google.com/github/supunabeywickrama/Transcript_Tool/blob/main/Transcript_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
import nbformat
print("Notebook metadata fix section removed.")

Notebook metadata fix section removed.


In [4]:
# ⬇️ System + Python deps
!apt -y install ffmpeg
!pip -q install faster-whisper==1.0.3 yt-dlp youtube-transcript-api==0.6.2 srt==3.5.3 webvtt-py==0.5.1 python-dotenv==1.0.1
# Optional cloud backend:
!pip -q install openai==1.52.2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 12.5 MB/s eta 0:00:00


In [5]:
# ⚙️ Configure where outputs will be saved
USE_DRIVE = True   # ← set False to keep outputs in /content/outputs

SAVE_DIR = "/content/outputs"
if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    SAVE_DIR = "/content/drive/MyDrive/transcripts"

import os
os.makedirs(SAVE_DIR, exist_ok=True)
SAVE_DIR


Mounted at /content/drive


'/content/drive/MyDrive/transcripts'

In [6]:
# 🧾 Helpers for JSON, SRT, VTT
import json
from typing import List, Dict

def _sec_to_timestamp(sec: float, srt: bool = True) -> str:
    h = int(sec // 3600); m = int((sec % 3600) // 60); s = sec % 60
    return (f"{h:02d}:{m:02d}:{s:06.3f}".replace(".", ",")
            if srt else f"{h:02d}:{m:02d}:{s:06.3f}".replace(",", "."))

def to_srt(segments: List[Dict]) -> str:
    lines = []
    for i, seg in enumerate(segments, 1):
        start = _sec_to_timestamp(seg["start"], srt=True)
        end   = _sec_to_timestamp(seg["end"],   srt=True)
        text  = seg["text"].strip()
        lines += [str(i), f"{start} --> {end}", text, ""]
    return "\n".join(lines)

def to_vtt(segments: List[Dict]) -> str:
    lines = ["WEBVTT", ""]
    for seg in segments:
        start = _sec_to_timestamp(seg["start"], srt=False)
        end   = _sec_to_timestamp(seg["end"],   srt=False)
        text  = seg["text"].strip()
        lines += [f"{start} --> {end}", text, ""]
    return "\n".join(lines)

def persist_outputs(basepath: str, result: Dict) -> Dict[str, str]:
    json_path = basepath + ".json"
    srt_path  = basepath + ".srt"
    vtt_path  = basepath + ".vtt"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    with open(srt_path, "w", encoding="utf-8") as f:
        f.write(to_srt(result["segments"]))
    with open(vtt_path, "w", encoding="utf-8") as f:
        f.write(to_vtt(result["segments"]))
    return {"json": json_path, "srt": srt_path, "vtt": vtt_path}


In [7]:
# 🧠 Local (offline) transcription with faster-whisper
from typing import Optional

def transcribe_local(audio_path: str, language: Optional[str] = None, model_size: str = "small") -> dict:
    from faster_whisper import WhisperModel
    model = WhisperModel(model_size, device="auto", compute_type="auto")
    segments_iter, info = model.transcribe(audio_path, language=language, vad_filter=True, beam_size=5)

    segments, full = [], []
    for seg in segments_iter:
        segments.append({"start": float(seg.start), "end": float(seg.end), "text": seg.text})
        full.append(seg.text)

    return {
        "backend":  "local",
        "model":    model_size,
        "language": info.language,
        "duration": sum(max(0.0, s["end"] - s["start"]) for s in segments),
        "segments": segments,
        "text":     " ".join(full).strip()
    }


In [8]:
# ☁️ Cloud transcription with OpenAI Whisper (optional)
# Set your key in a cell:   import os; os.environ["OPENAI_API_KEY"]="sk-..."
from typing import Optional

def transcribe_openai(audio_path: str, language: Optional[str] = None) -> dict:
    import os
    from openai import OpenAI
    if not os.environ.get("OPENAI_API_KEY"):
        raise RuntimeError("Set OPENAI_API_KEY env var to use OpenAI backend.")
    client = OpenAI()
    with open(audio_path, "rb") as f:
        resp = client.audio.transcriptions.create(model="whisper-1", file=f, language=language)
    text = resp.text.strip()
    return {
        "backend":  "openai",
        "model":    "whisper-1",
        "language": language or "",
        "duration": 0.0,
        "segments": [{"start": 0.0, "end": 0.0, "text": text}],
        "text":     text
    }


In [9]:
# ▶️ YouTube: try official captions first; otherwise download audio & transcribe
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
import yt_dlp, os
from typing import List, Dict, Optional

def try_fetch_youtube_captions(url: str, preferred_langs=("en", "en-US", "auto")) -> Optional[List[Dict]]:
    try:
        # yt_dlp resolves the ID robustly
        ydl_opts = {"quiet": True, "skip_download": True}
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            video_id = info.get("id")
        if not video_id:
            return None

        for lang in preferred_langs:
            try:
                return YouTubeTranscriptApi.get_transcript(video_id, languages=[lang])
            except Exception:
                continue

        transcripts = YouTubeTranscriptApi.list_transcripts(video_id)
        for tr in transcripts:
            try:
                return tr.fetch()
            except Exception:
                pass
    except (TranscriptsDisabled, NoTranscriptFound, Exception):
        return None
    return None

def download_youtube_audio(url: str, outdir: str) -> str:
    os.makedirs(outdir, exist_ok=True)
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": os.path.join(outdir, "%(id)s.%(ext)s"),
        "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": "mp3", "preferredquality": "192"}],
        "quiet": True,
        "nocheckcertificate": True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        return os.path.join(outdir, f"{info['id']}.mp3")


In [10]:
# 🧩 High-level wrappers to produce JSON/SRT/VTT into SAVE_DIR
from datetime import datetime
import tempfile, shutil, os

def transcribe_file_to_dir(in_path: str, save_dir: str, backend: str = "local",
                           language: Optional[str] = None, model_size: str = "small") -> dict:
    if backend == "openai":
        result = transcribe_openai(in_path, language)
    else:
        result = transcribe_local(in_path, language, model_size=model_size)

    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base  = os.path.join(save_dir, f"transcript_{stamp}")
    paths = persist_outputs(base, result)
    return {"paths": paths, "result": result}

def transcribe_youtube_to_dir(url: str, save_dir: str, backend: str = "local",
                              language: Optional[str] = None, model_size: str = "small") -> dict:
    caps = try_fetch_youtube_captions(url)
    if caps:
        segments = []
        for c in caps:
            start = float(c["start"]); end = start + float(c["duration"])
            segments.append({"start": start, "end": end, "text": c["text"]})
        result = {
            "backend":  "youtube_captions",
            "model":    "captions",
            "language": language or "",
            "duration": sum(max(0.0, s["end"] - s["start"]) for s in segments),
            "segments": segments,
            "text":     " ".join(s["text"] for s in segments).strip()
        }
        stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base  = os.path.join(save_dir, f"transcript_{stamp}")
        paths = persist_outputs(base, result)
        return {"paths": paths, "result": result}
    else:
        with tempfile.TemporaryDirectory() as tmp:
            audio = download_youtube_audio(url, tmp)
            return transcribe_file_to_dir(audio, save_dir, backend=backend, language=language, model_size=model_size)


In [11]:
# 📤 Upload any audio/video file (mp3, wav, mp4, mkv...) and transcribe
from google.colab import files
from typing import List, Dict, Tuple, Optional
import os

def _fmt_mmss(sec: float) -> str:
    if sec < 0: sec = 0.0
    m = int(sec // 60)
    s = int(round(sec % 60))
    return f"{m:02d}:{s:02d}"

def _group_segments_into_paragraphs(
    segments: List[Dict],
    max_gap_s: float = 1.0,         # if silence gap > this, start new paragraph
    max_chars: int = 350,           # soft cap per paragraph
    max_duration_s: float = 25.0    # hard cap on paragraph length
) -> List[Tuple[float, float, str]]:
    """
    Returns a list of (start_sec, end_sec, paragraph_text).
    """
    paras: List[Tuple[float, float, str]] = []
    if not segments:
        return paras

    cur_start = float(segments[0]["start"])
    cur_end   = float(segments[0]["end"])
    cur_texts = [segments[0]["text"].strip()]
    cur_chars = len(cur_texts[0])

    prev_end = cur_end

    for seg in segments[1:]:
        s = float(seg["start"])
        e = float(seg["end"])
        t = seg["text"].strip()

        gap = max(0.0, s - prev_end)
        next_chars = cur_chars + (1 if cur_texts else 0) + len(t)
        next_duration = max(cur_end, e) - cur_start

        # Decide if we should start a new paragraph
        if (gap > max_gap_s) or (next_chars > max_chars) or (next_duration > max_duration_s):
            # flush current paragraph
            paras.append((cur_start, cur_end, " ".join(cur_texts).strip()))
            # start new
            cur_start = s
            cur_end   = e
            cur_texts = [t]
            cur_chars = len(t)
        else:
            # extend current
            cur_end   = max(cur_end, e)
            cur_texts.append(t)
            cur_chars = next_chars

        prev_end = e

    # flush last
    paras.append((cur_start, cur_end, " ".join(cur_texts).strip()))
    return paras

def _save_paragraphs_txt(paragraphs: List[Tuple[float, float, str]], basepath_no_ext: str) -> str:
    out_txt = basepath_no_ext + ".txt"
    with open(out_txt, "w", encoding="utf-8") as f:
        for (st, en, txt) in paragraphs:
            f.write(f"{_fmt_mmss(st)}–{_fmt_mmss(en)} — {txt}\n\n")
    return out_txt

uploaded = files.upload()  # Pick one file
chosen = next(iter(uploaded)) if uploaded else None

# Settings
backend    = "local"      # "local" (offline) or "openai" (cloud)
language   = None         # e.g., "en" / "si" / "ta" (optional)
model_size = "small"      # tiny | base | small | medium | large-v3 (local only)

if chosen:
    out = transcribe_file_to_dir(chosen, SAVE_DIR, backend=backend, language=language, model_size=model_size)
    paths = out["paths"]
    result = out["result"]
    segments = result.get("segments", [])

    # Build paragraphs
    paragraphs = _group_segments_into_paragraphs(
        segments,
        max_gap_s=1.2,        # tweak if you want bigger/smaller paragraphs
        max_chars=380,
        max_duration_s=30.0
    )

    # Print full transcript as paragraphs with time ranges
    print("========== FULL TRANSCRIPT (paragraphs with timestamps) ==========\n")
    for (st, en, txt) in paragraphs:
        print(f"{_fmt_mmss(st)}–{_fmt_mmss(en)} — {txt}\n")

    # Save a .txt version alongside JSON/SRT/VTT
    base_no_ext = os.path.splitext(paths["json"])[0]  # use same base name
    txt_path = _save_paragraphs_txt(paragraphs, base_no_ext)

    print("===============================================================")
    print("Saved files:")
    print(paths)                         # JSON / SRT / VTT
    print({"txt": txt_path})             # Paragraphs TXT
else:
    print("No file selected.")



Saving step_2.mp4 to step_2 (2).mp4


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

========== FULL TRANSCRIPT (paragraphs with timestamps) ==========

00:00–00:25 — Hello, and welcome back to our series. In this video, we'll cover how to add overlays on top of our rendered images. Overlays are highlights that signal interactivity, making certain areas appear clickable to the user. Think about it like point-and-click games. Monkey Island, for example, was one of my favorites. Let's start with a quick recap on the previous episode.

00:25–00:50 — You identified which room you want to model. You got your floor plan, or you simply measured it yourself. You modeled your room in your preferred software. In my case, I use SketchUp. You can also model in Sweet Home 3D. And let me know in the comments if you are going to do an episode on Sweet Home 3D. And finally, you exported two renders from your scene. Awesome.

00:50–01:19 — So let's now talk about some overlay types. For the lampshade example, what is really happening here is that we are manipulating an image of the lam

 # **Transcribe a YouTube link**

In [12]:
# 🔗 Paste a YouTube URL (with or without captions)
youtube_url = "https://www.youtube.com/watch?v=vYqP-3ijl-8"  # e.g., "https://www.youtube.com/watch?v=xxxxxxxxxxx"

backend    = "local"   # "local" (offline) or "openai" (cloud, usually no per-segment times)
language   = None      # e.g., "en" / "si" / "ta"
model_size = "small"   # tiny | base | small | medium | large-v3 (local only)

# ---------- helpers for paragraph view ----------
from typing import List, Dict, Tuple, Optional
import os

def _fmt_mmss(sec: float) -> str:
    if sec < 0: sec = 0.0
    m = int(sec // 60)
    s = int(round(sec % 60))
    return f"{m:02d}:{s:02d}"

def _group_segments_into_paragraphs(
    segments: List[Dict],
    max_gap_s: float = 1.2,       # silence gap to start a new paragraph
    max_chars: int = 380,         # soft cap on paragraph text length
    max_duration_s: float = 30.0  # hard cap on paragraph time span
) -> List[Tuple[float, float, str]]:
    """
    Returns a list of (start_sec, end_sec, paragraph_text).
    Works with segments from YouTube captions OR local transcription.
    """
    out: List[Tuple[float, float, str]] = []
    if not segments:
        return out

    cur_start = float(segments[0]["start"])
    cur_end   = float(segments[0]["end"])
    cur_texts = [segments[0]["text"].strip()]
    cur_chars = len(cur_texts[0])
    prev_end  = cur_end

    for seg in segments[1:]:
        s = float(seg["start"]); e = float(seg["end"]); t = seg["text"].strip()
        gap = max(0.0, s - prev_end)
        next_chars = cur_chars + (1 if cur_texts else 0) + len(t)
        next_duration = max(cur_end, e) - cur_start

        if (gap > max_gap_s) or (next_chars > max_chars) or (next_duration > max_duration_s):
            out.append((cur_start, cur_end, " ".join(cur_texts).strip()))
            cur_start, cur_end, cur_texts, cur_chars = s, e, [t], len(t)
        else:
            cur_end = max(cur_end, e)
            cur_texts.append(t)
            cur_chars = next_chars

        prev_end = e

    out.append((cur_start, cur_end, " ".join(cur_texts).strip()))
    return out

def _save_paragraphs_txt(paragraphs: List[Tuple[float, float, str]], basepath_no_ext: str) -> str:
    out_txt = basepath_no_ext + ".txt"
    with open(out_txt, "w", encoding="utf-8") as f:
        for (st, en, txt) in paragraphs:
            f.write(f"{_fmt_mmss(st)}–{_fmt_mmss(en)} — {txt}\n\n")
    return out_txt

# ---------- run ----------
if youtube_url.strip():
    out = transcribe_youtube_to_dir(
        youtube_url.strip(), SAVE_DIR,
        backend=backend, language=language, model_size=model_size
    )
    paths   = out["paths"]
    result  = out["result"]
    segs    = result.get("segments", [])

    # Build and print paragraphs
    paragraphs = _group_segments_into_paragraphs(
        segs, max_gap_s=1.2, max_chars=380, max_duration_s=30.0
    )

    print("========== FULL TRANSCRIPT (paragraphs with timestamps) ==========\n")
    for (st, en, txt) in paragraphs:
        print(f"{_fmt_mmss(st)}–{_fmt_mmss(en)} — {txt}\n")

    # Save a .txt next to JSON/SRT/VTT
    base_no_ext = os.path.splitext(paths["json"])[0]
    txt_path = _save_paragraphs_txt(paragraphs, base_no_ext)

    print("===============================================================")
    print("Saved files:")
    print(paths)             # JSON / SRT / VTT
    print({"txt": txt_path}) # Paragraphs TXT
else:
    print("Set youtube_url and run this cell.")


         player = https://www.youtube.com/s/player/e237d4c5/player_ias.vflset/en_US/base.js
         n = N9BOeSLI6IOaLjeUbDf ; player = https://www.youtube.com/s/player/e237d4c5/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
         player = https://www.youtube.com/s/player/e237d4c5/player_ias.vflset/en_US/base.js
         n = s0UMHgA9jmgRW0JxW7l ; player = https://www.youtube.com/s/player/e237d4c5/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


========== FULL TRANSCRIPT (paragraphs with timestamps) ==========

00:00–00:19 — Russia's mounting for Andrew Mountbatten-Windsor, formally Prince Andrew, to give evidence in the United States about his former friend, the late convicted pedophile Jeffrey Epstein. While victims of Epstein, as well as several members of Congress, have called for the former prince to speak to the authorities to tell them exactly what he knew and when.

00:19–00:37 — While Andrew, who's now been stripped of all royal titles, has always denied any personal wrongdoing, John Sudworth has the latest on that. Following his history-making announcement, photographers captured the first glimpse of King Charles driving on his Sandringham estate, while for now his younger brother remains

00:37–01:01 — here at Windsor's Royal Lodge, the home he soon to lose, along with all his titles. But far from drawing a line in the US, it's led to renewed calls for Andrew to reveal all he knows about his friend and convicted pe

# **Batch mode (many files/links at once)**

In [11]:
# 📦 Process a list of local paths and/or YouTube URLs
items = [
    # "/content/your_audio.mp3",
    # "https://www.youtube.com/watch?v=xxxxxxxxxxx",
]

backend    = "local"   # or "openai"
language   = None
model_size = "small"

for item in items:
    try:
        if item.startswith("http"):
            out = transcribe_youtube_to_dir(item, SAVE_DIR, backend=backend, language=language, model_size=model_size)
        else:
            out = transcribe_file_to_dir(item, SAVE_DIR, backend=backend, language=language, model_size=model_size)
        print("Saved:", out["paths"])
    except Exception as e:
        print("Failed:", item, "→", e)
